## Import

In [155]:
import os
import time
import glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# sklearn
from sklearn.metrics import log_loss, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import random

# tensorflow
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

## Data Load

In [156]:
info = pd.read_csv('./data/snp_info.csv')
info

,SNP_id,name,chrom,cm,pos
0,SNP_01,BTA-19852-no-rs,2,67.05460,42986890
1,SNP_02,ARS-USMARC-Parent-DQ647190-rs29013632,6,31.15670,13897068
2,SNP_03,ARS-BFGL-NGS-117009,6,68.28920,44649549
3,SNP_04,ARS-BFGL-NGS-60567,6,77.87490,53826064
4,SNP_05,BovineHD0600017032,6,80.50150,61779512
5,SNP_06,BovineHD0600017424,6,80.59540,63048481
6,SNP_07,Hapmap49442-BTA-111073,6,80.78000,64037334
7,SNP_08,BovineHD0600018638,6,82.68560,67510588
8,SNP_09,ARS-BFGL-NGS-37727,6,86.87400,73092782
9,SNP_10,BTB-01558306,7,62.06920,40827112


In [157]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [158]:
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


In [159]:
train['father'].sum(), train['mother'].sum(), train['gender'].sum()

(0, 0, 0)

In [160]:
train['father'].unique(), train['mother'].unique(), train['gender'].unique()

(array([0]), array([0]), array([0]))

In [161]:
test

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,TEST_000,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,TEST_001,0,0,0,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,TEST_002,0,0,0,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G
3,TEST_003,0,0,0,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A
4,TEST_004,0,0,0,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,TEST_170,0,0,0,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A
171,TEST_171,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A
172,TEST_172,0,0,0,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G
173,TEST_173,0,0,0,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A


In [162]:
test['father'].sum(), test['mother'].sum(), test['gender'].sum()

(0, 0, 0)

In [163]:
test['father'].unique(), test['mother'].unique(), test['gender'].unique()

(array([0]), array([0]), array([0]))

In [164]:
train = train.drop(columns=['id', 'father', 'mother', 'gender'])
test = test.drop(columns=['id', 'father', 'mother', 'gender'])


## Data Pre-processing
### Label-Encoding

In [165]:
TARLE = LabelEncoder()
SNPLE = LabelEncoder()
SNPCOL = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [166]:
SNPCOL

['SNP_01',
 'SNP_02',
 'SNP_03',
 'SNP_04',
 'SNP_05',
 'SNP_06',
 'SNP_07',
 'SNP_08',
 'SNP_09',
 'SNP_10',
 'SNP_11',
 'SNP_12',
 'SNP_13',
 'SNP_14',
 'SNP_15']

In [167]:
snp_data = []
for col in SNPCOL:
    snp_data += list(train[col].values)

In [168]:
train.iloc[:,-1] = TARLE.fit_transform(train.iloc[:,-1])
SNPLE.fit(snp_data)

LabelEncoder()

In [169]:
train

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,1
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,2
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,1
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,0
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,A G,A G,A A,G A,C C,A G,A A,G A,A A,G G,A G,G A,A A,A A,A A,1
258,2,G G,A A,C A,A A,A A,A G,G A,G A,A A,A G,A G,A A,A G,A A,G A,2
259,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A A,G G,G G,G G,C A,G G,0
260,1,A A,G G,A A,G A,A A,G G,G G,A A,G A,A G,A G,G A,G G,C A,G G,0


In [170]:
for col in train.columns:
    if col in SNPCOL:
        train[col] = SNPLE.transform(train[col])
        test[col] = SNPLE.transform(test[col])

In [171]:
# provides train/test indices to split data in train/test sets.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)



for train_idx, test_idx in split.split(train, train.iloc[:,0]):

    x_train = train.iloc[:,:-1].loc[train_idx]

    x_test = train.iloc[:,:-1].loc[test_idx]

    y_train = train.iloc[:,-1].loc[train_idx]

    y_test = train.iloc[:,-1].loc[test_idx]

In [172]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [173]:
model = Sequential()
model.add(Dense(128,activation='elu',input_dim = 16))
model.add(Dense(64,activation='elu'))
model.add(Dense(32,activation='elu'))
model.add(Dense(16,activation='elu'))
model.add(Dropout(0.4275))
model.add(Dense(3,activation='softmax'))

In [174]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, mode='min',
#                               patience=20, min_lr=1e-8)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'],
              loss=keras.losses.categorical_crossentropy)
# ,
#               metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
#                         tfa.metrics.F1Score(num_classes=3, average='macro')]


history = model.fit(x_train, y_train, epochs=150, batch_size=32, validation_data=([x_test, y_test]))
# , callbacks=[reduce_lr]



Epoch 1/150
7/7 [==============================] - 1s 21ms/step - loss: 1.4672 - accuracy: 0.3301 - val_loss: 1.1174 - val_accuracy: 0.4340
Epoch 2/150
7/7 [==============================] - 0s 4ms/step - loss: 1.2833 - accuracy: 0.3349 - val_loss: 1.0395 - val_accuracy: 0.4151
Epoch 3/150
7/7 [==============================] - 0s 4ms/step - loss: 1.1156 - accuracy: 0.4019 - val_loss: 0.9705 - val_accuracy: 0.5094
Epoch 4/150
7/7 [==============================] - 0s 5ms/step - loss: 1.0329 - accuracy: 0.4833 - val_loss: 0.9087 - val_accuracy: 0.6038
Epoch 5/150
7/7 [==============================] - 0s 5ms/step - loss: 0.9875 - accuracy: 0.4976 - val_loss: 0.8480 - val_accuracy: 0.6415
Epoch 6/150
7/7 [==============================] - 0s 4ms/step - loss: 0.8806 - accuracy: 0.5933 - val_loss: 0.7873 - val_accuracy: 0.6792
Epoch 7/150
7/7 [==============================] - 0s 5ms/step - loss: 0.7883 - accuracy: 0.6507 - val_loss: 0.7329 - val_accuracy: 0.6981
Epoch 8/150
7/7 [=========

In [175]:
submit = pd.read_csv('./data/sample_submission.csv')

In [176]:
preds = model.predict(test)
print('Done.')

6/6 [==============================] - 0s 785us/step
Done.


In [177]:
submit['class'] = TARLE.inverse_transform(np.argmax(preds, axis=1))

In [178]:
t = pd.read_csv('./trial 14 (99.06%).csv')
check = []
for idx in range(len(submit)):
    if submit['class'][idx] == t['class'][idx]: check.append(True)
    else: check.append(False)
pd.DataFrame(check).value_counts()

True     171
False      4
dtype: int64

In [179]:
t = pd.read_csv('./trial 24 (98.17%).csv')
check = []
for idx in range(len(submit)):
    if submit['class'][idx] == t['class'][idx]: check.append(True)
    else: check.append(False)
pd.DataFrame(check).value_counts()

True     173
False      2
dtype: int64

In [180]:
t = pd.read_csv('./trial 16 (98.14%).csv')
check = []
for idx in range(len(submit)):
    if submit['class'][idx] == t['class'][idx]: check.append(True)
    else: check.append(False)
pd.DataFrame(check).value_counts()

True     172
False      3
dtype: int64

In [181]:
t = pd.read_csv('./trial 15 (98.11%).csv')
check = []
for idx in range(len(submit)):
    if submit['class'][idx] == t['class'][idx]: check.append(True)
    else: check.append(False)
pd.DataFrame(check).value_counts()

True     171
False      4
dtype: int64

In [182]:
# submit.to_csv('./trial 25.csv', index=False)